In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
batch_size = 100

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root="./mnist_data/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="./mnist_data/", train=False, transform=transforms.ToTensor(), download=False)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False)

In [4]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()

        ######## Encoder part ########
        # common hidden layer
        self.fc_common_encode = nn.Sequential(
            nn.Linear(x_dim, h_dim1),
            nn.ReLU(),
            nn.Dropout(p=0.2),

            nn.Linear(h_dim1, h_dim2),
            nn.ReLU(),
            nn.Dropout(p=0.2)
        )

        # mu layer
        self.fc_mu = nn.Linear(h_dim2, z_dim)

        # log_var layer
        self.fc_logvar = nn.Linear(h_dim2, z_dim)
        ##############################

        ######## Decoder part ########
        # 1st hidden layer
        self.fc_decode = nn.Sequential(
            nn.Linear(z_dim, h_dim2),
            nn.ReLU(),
            nn.Dropout(p=0.2),

            nn.Linear(h_dim2, h_dim1),
            nn.ReLU(),
            nn.Dropout(p=0.2),

            nn.Linear(h_dim1, x_dim)
        )
        ##############################

    def encoder(self, x):
        h = self.fc_common_encode(x)
        mu = self.fc_mu(h)
        log_var = self.fc_logvar(h)

        return mu, log_var

    def sampling(self, mu, log_var):
        std = torch.exp(log_var / 2)
        epsilon = torch.randn_like(std)

        return mu + epsilon * std

    def decoder(self, z):
        x = F.sigmoid(self.fc_decode(z))
        return x

    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)

        return self.decoder(z), mu, log_var


In [5]:
vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [6]:
vae

VAE(
  (fc_common_encode): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
  )
  (fc_mu): Linear(in_features=256, out_features=2, bias=True)
  (fc_logvar): Linear(in_features=256, out_features=2, bias=True)
  (fc_decode): Sequential(
    (0): Linear(in_features=2, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=784, bias=True)
  )
)

In [7]:
optimizer = optim.Adam(vae.parameters())

# return reconstruction error + Regularization
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = 0.5 * torch.sum(mu.pow(2) + log_var.exp() - log_var - 1)

    # print(f"bce: {BCE}, KLD: {KLD}")

    return BCE + KLD

In [8]:
def train(epoch):
    vae.train()
    train_loss = 0

    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()

        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')

    print(f'====> Epoch: {epoch} Average loss: {train_loss/len(train_loader.dataset):.4f}')

In [9]:
def test():
    vae.eval()

    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    print(f'====> Test set loss: {test_loss:.4f}')

In [10]:
for epoch in range(1, 11):
    train(epoch)
    test()

Train epoch: 1 [0/60000 (0%)]	Loss: 546.006172
Train epoch: 1 [10000/60000 (17%)]	Loss: 194.714590
Train epoch: 1 [20000/60000 (33%)]	Loss: 173.537871
Train epoch: 1 [30000/60000 (50%)]	Loss: 178.276094
Train epoch: 1 [40000/60000 (67%)]	Loss: 166.352891
Train epoch: 1 [50000/60000 (83%)]	Loss: 168.311855
====> Epoch: 1 Average loss: 182.6296
====> Test set loss: 161.6622
Train epoch: 2 [0/60000 (0%)]	Loss: 162.587744
Train epoch: 2 [10000/60000 (17%)]	Loss: 165.567539
Train epoch: 2 [20000/60000 (33%)]	Loss: 154.001084
Train epoch: 2 [30000/60000 (50%)]	Loss: 154.884473
Train epoch: 2 [40000/60000 (67%)]	Loss: 149.089121
Train epoch: 2 [50000/60000 (83%)]	Loss: 167.031465
====> Epoch: 2 Average loss: 161.5395
====> Test set loss: 155.5901
Train epoch: 3 [0/60000 (0%)]	Loss: 159.076807
Train epoch: 3 [10000/60000 (17%)]	Loss: 155.021641
Train epoch: 3 [20000/60000 (33%)]	Loss: 154.307744
Train epoch: 3 [30000/60000 (50%)]	Loss: 151.800234
Train epoch: 3 [40000/60000 (67%)]	Loss: 160.04

In [11]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()

    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')